Start by creating a new Notebook for this assignment.

In [8]:
#imports
from bs4 import BeautifulSoup
import requests
#Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

Find the table in the scraped data

In [9]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

Append each row in the table

In [10]:
A=[]
B=[]
C=[]

for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

Put data in pandas dataframe

In [11]:
import pandas as pd
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C
#df.head()

Clean data by removing new line at end of most rows

In [12]:
df.replace(r'\n', '', regex = True, inplace = True)
#df.head()

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [13]:
df.drop(df[(df['Borough'] == "Not assigned")].index, inplace=True)
df = df.reset_index(drop=True)
#df.head()

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [14]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
#df.head()

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [15]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
#df

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [16]:
df.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

In [17]:
#imports
import requests
import io

s=requests.get("http://cocl.us/Geospatial_data").content
df1=pd.read_csv(io.StringIO(s.decode('utf-8')))

Renaming Postal code column to Postcode

In [18]:
df1.columns = ['Postcode', 'Latitude', 'Longitude']

Merging the two dataframes

In [19]:
df = pd.merge(df1, df, on='Postcode')

Rearrange and display new dataframe

In [20]:
df = df[['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]

Find the Lat and Long of Toronto

In [26]:
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Generate map with Folium

In [30]:
import folium

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Could not get Foursquare to function, unable to further analyse neighbourhoods.